<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 分類のためのファインチューニング
- LLMのさまざまなファインチューニングアプローチ
- スパムメールを識別するために事前学習済みLLMをファインチューニングする

## 6.1 ファインチューニングのさまざまなカテゴリ
- インストラクションチューニング: 特定の指示を使用した一連のタスクを言語モデルに訓練することで、自然言語のプロンプトで表示されたタスクを理解して実行する能力を向上させる
- 分類チューニング: 特定のクラスラベルを認識する能力を向上させる

インストラクションチューニングを行ったモデルは、幅広いタスクに対応できる

分類チューニングを行ったモデルは、訓練中に遭遇したクラスの予測に限定される（専門性が高い）

## 6.2 データセットを準備する

In [ ]:
# データセットのダウンロードと解凍
import urllib.request
import zipfile
import os
from pathlib import Path

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction")
        return

    with urllib.request.urlopen(url) as response:
        # ファイルダウンロード
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        # ファイル解凍
        zip_ref.extractall(extracted_path)

    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path) # ファイル拡張子.tsvを追加
    print(f"File downloaded and saved as {data_file_path}")

download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

In [ ]:
# データの読み込み
import pandas as pd

df = pd.read_csv(
    data_file_path, sep="\t", header=None, names=["Label", "Text"]
)

df

In [ ]:
# クラスラベルの分布
print(df["Label"].value_counts())

In [ ]:
# アンダーサンプリングして均衡なデータセットを作成
def create_balanced_dataset(df):
    # スパムの数に合わせてデータセットをアンダーサンプリング
    num_spam = df[df["Label"] == "spam"].shape[0]
    ham_subset = df[df["Label"] == "ham"].sample(
        num_spam, random_state=123
    )
    balanced_df = pd.concat(
        [ham_subset, df[df["Label"] == "spam"]]
    )

    return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

In [ ]:
# ラベルのマッピング
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})

In [ ]:
# データセットを訓練／検証／評価用に分割する
def random_split(df, train_frac, validation_frac):

    df = df.sample(
        frac=1, random_state=123
    ).reset_index(drop=True)
    train_end = int(len(df) * train_frac) # 分割インデックスを計算
    validation_end = train_end + int(len(df) * validation_frac)

    train_df = df[:train_end] # DataFrameを分割
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)

In [ ]:
# CSVファイルで保存
train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

## 6.3 データローダーを作成する